## Introduction

This chapter addresses three major topics that are closely related:
- Python's `async def`, `await`, `async with`, and `async for` constructs
- Objects supporting those constructs: native coroutines and asynchronous variants of context managers, iterables, generators, and comprehensions
- `asyncio` and other asynchronous libraries

## A Few Definitions
- <span style="color:skyblue">***Native coroutine***</span>: A coroutine function defined with `async def`. You can delegate from a native coroutine to another native coroutine using the `await` keyword, similar to how classic coroutines use `yield` from. The `async def` statement always defines a native coroutine, even if the await keyword is not used in its body. The `await` keyword cannot be used outside of a native coroutine
- <span style="color:skyblue">***Classic coroutine***</span>: A generator function that consumes data sent to it via `my_coro.send(data)` calls, and reads that data by using `yield` in an expression. Classic coroutines can delegate to other classic coroutines using `yield` from. Classic coroutines cannot be driven by `await`, and are no longer supported by `asyncio`.
- <span style="color:skyblue">***Generator-based coroutine***</span>: A generator function decorated with `@types.coroutine` — introduced in Python 3.5. That decorator makes the generator compatible with the new `await` keyword.
- <span style="color:skyblue">***Asynchronous generator***</span>: A generator function defined with `async def` and using `yield` in its body. It returns an asynchronous generator object that provides `__anext__`, a coroutine method to retrieve the next item.

## An `asyncio` Example: Probing Domains

We want to write a script (`blogdom.py`) to concurrently check if domains `<name>.dev` are available for our Python blog. The output should be  
```bash
python3 blogdom.py
with.dev
+ elif.dev
+ def.dev
from.dev
```
Compared to checking domains sequentially, probing DNS via native coroutine objects should be much faster, almost the same as the time to check the single slowest DNS, instead of the sum of all responses

In [1]:
import asyncio
import socket
from keyword import kwlist
import nest_asyncio

nest_asyncio.apply()  # allow nested usage of asyncio. Only needs for jupyter notebook

MAX_KEYWORD_LEN = 4  # Max length of keyword for domains, because shorter is better.


async def probe(domain: str) -> tuple[str, bool]:
    """
    Returns a tuple with the domain name and a boolean; True means the domain resolved
    Returning the domain name will make it easier to display the results
    """
    loop = asyncio.get_running_loop()  # Get a reference to the asyncio event loop, so we can use it next.
    try:
        await loop.getaddrinfo(domain, None)  # loop.getaddrinfo coroutine returns a five-part tuple of
                                              # parameters to connect to the given address using a socket
                                              # In this example, we don’t need the result. 
                                              # If we got it, the domain resolves; otherwise, it doesn’t.
    except socket.gaierror:  # `gai` stands for get address info 
        return (domain, False)
    return (domain, True)


async def main() -> None:  # main must be a coroutine, so that we can use await in it.
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)  # Generator to yield Python 
                                                                 # keywords with length up to `MAX_KEYWORD_LEN`
    domains = (f'{name}.dev'.lower() for name in names)  # Generator to yield domain names with the `.dev` suffix
    coros = [probe(domain) for domain in domains]  # Build a list of coroutine objects
    for coro in asyncio.as_completed(coros):  # `asyncio.as_completed` is a generator that yields 
                                              # coroutines that return the results of the 
                                              # coroutines passed to it in the order they are completed
        domain, found = await coro  # At this point, we know the coroutine is done 
                                    # because that’s how as_completed works
        mark = '+' if found else ' '
        print(f'{mark} {domain}')


asyncio.run(main())  # starts the event loop and returns only when the event loop exits.

+ as.dev
+ in.dev
+ try.dev
+ def.dev
+ from.dev
+ and.dev
  is.dev
  true.dev
  with.dev
  elif.dev
  pass.dev
  for.dev
  if.dev
+ not.dev
  none.dev
  or.dev
  else.dev
+ del.dev


**Low-level Explainations:**

<span style="color:lightgreen">***Using the syntax `await loop.getaddrinfo(...)` avoids blocking because `await` suspends the current coroutine object. For example, during the execution of the `probe('if.dev')` coroutine, a new coroutine object is created by `getaddrinfo('if.dev', None)`. Awaiting it starts the low-level `addrinfo` query and yields control back to the event loop, not to the `probe(‘if.dev’)` coroutine, which is suspended. The event loop can then drive other pending coroutine objects, such as `probe('or.dev')`. When the event loop gets a response for the `getaddrinfo('if.dev', None)` query, that specific coroutine object resumes and returns control back to the `probe('if.dev')` — which was suspended at `await` — and can now handle a possible exception and return the result tuple.***</span>

## Guido's Trick to Read Async Code

<span style="color:skyblue">***The trick is to pretend that the `async` and `await` keywords are not there. If you do that, you’ll realize that coroutines read like plain old sequential functions, but they just magically never block.***</span>

## Awaitables

<span style="color:skyblue">*The `for` keyword works with iterables. The `await` keyword works with awaitables.*</span> As an end user of `asyncio`, these are the awaitables you will see on a daily basis:  
- <span style="color:skyblue">*A native coroutine object, which you get by calling a native coroutine function. We use `await other_coro()` to run `other_coro` right now and `wait` for its completion because we need its result before we can proceed.*</span>
- <span style="color:skyblue">*An `asyncio.Task`, which you usually get by passing a coroutine object to `asyncio.create_task()`. However, end-user code does not always need to `await` on a `Task`. We use `asyncio.create_task(one_coro())` to schedule `one_coro` for concurrent execution, without waiting for its return.*</span>

Awaitables are 
- Objects with an `__await__` method that returns an iterator, e.g. an `asyncio.Future` instance
- Objects written in other languages using the Python / C API with a `tp_as_async.am_await` function, returning an iterator

## Downloading with `asyncio` and `HTTPX`

Look into the code of `flags_asyncio`:

```python
import asyncio

from httpx import AsyncClient

from flags import BASE_URL, save_flag, main


async def download_one(client: AsyncClient, cc: str):
    """
    `download_one` must be a native coroutine, so it can await on `get_flag` — 
    which does the HTTP request. Then it displays the code of the downloaded flag, and
    saves the image.
    """
    image = await get_flag(client, cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc


async def get_flag(client: AsyncClient, cc: str) -> bytes:
    """
    get_flag needs to receive the AsyncClient to make the request.
    """
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = await client.get(url, timeout=6.1,
                follow_redirects=True)  # The get method of an httpx.AsyncClient 
                                        # instance returns a ClientResponse
                                        # object that is also an asynchronous context manager
    return resp.read()  # Network I/O operations are implemented as coroutine methods, so they are
                        # driven asynchronously by the asyncio event loop
```

The code delegates to the `httpx` coroutines explicitly through `await` or implicitly through the special methods of the asynchronous context managers, such as `AsyncClient` and `ClientResponse`

## The Secret of Native Coroutines: Humble Generators

Using `httpx`, we don't need to use `.send()` calls or `yield` expressions. The code sits beteen the `asyncio` library and the async libraries we are using, e.g. `httpx`

<img src="../images/async.png" style="width: 50%;">.  

Under the hood, the asyncio event loop makes the `.send` calls that drive your coroutines, and your coroutines `await` on other coroutines, including library coroutines. Using functions like `asyncio.gather` and `asyncio.create_task`, you can start multiple concurrent await channels, enabling concurrent execution of multiple I/O operations driven by a single event loop, in a single thread.

## The All-or-Nothing Problem

<span style="color:skyblue">***For peak performance with `asyncio`, we must replace every function that does I/O with an asynchronous version that is activated with `await` or `asyncio.create_task`, so that control is given back to the event loop while the function waits for I/O. If you can’t rewrite a blocking function as a coroutine, you should run it in a separate thread or process.***</span>

<span style="color:orange">***“You rewrite all your code so none of it blocks or you’re just wasting your time.”***</span>. This is why in `flags_asyncio.py`, all functions (except for `download_many` which calls `asyncio.run`) are coroutines